<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=2a1b9bcef43e271f27b83fd4243c236150d76827c0888c2948ad0cf8bab7f45c
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf


    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-10 09:15:10
-------------------
qualified stocks: 87
with latest results: 26
still star stocks: 16
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  1.05 L
Current:  1.43 C
-------------------
Today PnL: 18.58 K (0.13%)
Current PnL: -21.59 L (-14.15%)
CY Booked + Current PnL: -8.94 L (-5.86%)
-------------------
Total profit:  2.19 L
Total loss:  -23.79 L
-------------------
Total Booked + Current PnL: 18.04 L (14.36%)
Total Booked PnL: 39.63 L (31.55%)
Curr Year Booked PnL: 12.65 L (8.85%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.32 C
Est FTT PnL: 88.57 L (61.95%)
Deployed:  1.26 C
Current:  1.43 C
CAGR/XIRR %: 7.54%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
58,RELIANCE,1533.00,-7.28,62.0,X-LC,6.72,231644.0,21278.0,7505.0,0.47,10.11,3.24,13.68,37.0,2.84,1.63,28.31,XY25,NTT,REFINERIES
79,UNIONBANK,163.00,-2.33,70.0,M-LC,11.57,175894.0,35054.0,9446.0,0.91,24.89,5.37,31.59,66.0,3.71,1.24,57.97,XY24,NTT,BANKS
78,TTKPRESTIG,770.00,102.13,52.0,M-SC,2.02,87115.0,-13662.0,13755.0,-0.50,-13.56,15.79,0.09,245.0,-0.99,0.61,14.45,OX40N,NTT,DURABLES
33,ICICIGI,2252.93,-13.14,59.0,X-MC,6.26,149495.0,13362.0,17222.0,0.00,9.82,11.52,22.47,91.0,0.78,1.05,24.07,X40,ATH,INSURANCE
2,ABBOTINDIA,35195.00,-11.87,37.0,X-MC,3.12,86970.0,-3708.0,18612.0,0.71,-4.09,21.40,16.44,101.0,-0.20,0.61,15.26,X40,ATH,PHARMA


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
46,KPIGREEN,731.05,9.02,57.0,H-SC,9.73,129780.0,4483.0,54443.0,2.34,3.58,41.95,47.03,141.0,0.08,0.91,62.50,MH,ATH,POWER
49,MASFIN,398.61,-17.80,53.0,H-SC,8.00,93300.0,-4680.0,26283.0,1.19,-4.78,28.17,22.05,152.0,-0.18,0.66,35.62,XR,ATH,FINANCE
32,HONAUT,58357.33,-22.26,49.0,X-SC,0.84,109485.0,-18453.0,65592.0,1.18,-14.42,59.91,36.84,143.0,-0.28,0.77,12.09,X40N,ATH,ELECTRICAL
55,RAJOOENG,143.10,-42.01,48.0,H-SC,16.55,87990.0,-14510.0,55108.0,1.08,-14.16,62.63,39.61,114.0,-0.26,0.62,7.46,AR,ATH,MISC
39,INFY,2275.00,-18.52,55.0,X-LC,4.90,317583.0,4695.0,166985.0,0.96,1.50,52.58,54.87,3.0,0.03,2.24,9.91,X40,BTT,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
10,BAJAJHFL,181.5,-16.38,35.0,X-MC,6.35,176333.0,-24173.0,121335.0,-1.81,-12.06,68.81,48.45,90.0,-0.20,1.24,1.68,X40N,ATH,FINANCE
1,ABB,7934.0,-42.98,32.0,H-LC,4.96,238272.0,-23347.0,142558.0,-1.05,-8.92,59.83,45.57,7.0,-0.16,1.68,1.39,AR,NTT,ELECTRICAL
71,SYMPHONY,1306.0,-45.13,40.0,M-SC,16.28,117121.0,-54020.0,53969.0,-0.82,-31.56,46.08,-0.03,196.0,-1.00,0.83,2.11,OX40N,NTT,DURABLES
38,INDUSINDBK,1800.0,267.65,58.0,L-MC,7.72,45087.0,-32120.0,57513.0,-0.78,-41.60,127.56,32.89,258.0,-0.56,0.32,24.17,XR,NTT,BANKS
9,AWL,485.0,-63.95,51.0,X-MC,2.22,256799.0,-44952.0,206364.0,-0.70,-14.90,80.36,53.50,116.0,-0.22,1.81,14.06,XY24,NTT,FMCG


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.0,102.13,52.0,M-SC,2.02,87115.0,-13662.0,13755.0,-0.50,-13.56,15.79,0.09,245.0,-0.99,0.61,14.45,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.97,42.0,H-SC,2.33,220500.0,-49167.0,85510.0,-0.54,-18.23,38.78,13.47,138.0,-0.57,1.55,12.12,XY24,NTT,PAINTS
17,CERA,9475.0,-24.34,42.0,H-SC,1.80,138023.0,-37880.0,79902.0,0.07,-21.53,57.89,23.89,149.0,-0.47,0.97,19.20,OX40N,NTT,CERAMICS
42,JCHAC,2282.0,19921.74,62.0,M-SC,1.51,97860.0,-29945.0,29935.0,-0.03,-23.43,30.59,-0.01,232.0,-1.00,0.69,10.81,OX40N,NTT,AC
67,SIS,528.0,1993.26,42.0,H-SC,2.41,84482.0,-26550.0,50157.0,-0.39,-23.91,59.37,21.26,156.0,-0.53,0.60,14.14,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,UNIONBANK,163.0,-2.33,70.0,M-LC,11.57,175894.0,35054.0,9446.0,0.91,24.89,5.37,31.59,66.0,3.71,1.24,57.97,XY24,NTT,BANKS


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAIL,228.00,54.77,66.0,M-MC,10.81,244007.0,19045.0,148600.0,0.50,8.47,60.90,74.53,192.0,0.13,1.72,43.45,XY24,BTT,STEEL
31,HINDZINC,730.22,27.24,49.0,M-LC,9.32,207495.0,2419.0,110159.0,0.53,1.18,53.09,54.89,52.0,0.02,1.46,25.56,X5K,ATH,METALS
46,KPIGREEN,731.05,9.02,57.0,H-SC,9.73,129780.0,4483.0,54443.0,2.34,3.58,41.95,47.03,141.0,0.08,0.91,62.50,MH,ATH,POWER
79,UNIONBANK,163.00,-2.33,70.0,M-LC,11.57,175894.0,35054.0,9446.0,0.91,24.89,5.37,31.59,66.0,3.71,1.24,57.97,XY24,NTT,BANKS
85,WHIRLPOOL,2270.00,-40.16,54.0,M-SC,7.14,124154.0,9356.0,84685.0,0.71,8.15,68.21,81.92,223.0,0.11,0.87,46.98,XR,NTT,DURABLES


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WHIRLPOOL,2270.00,-40.16,54.0,M-SC,7.14,124154.0,9356.0,84685.0,0.71,8.15,68.21,81.92,223.0,0.11,0.87,46.98,XR,NTT,DURABLES
46,KPIGREEN,731.05,9.02,57.0,H-SC,9.73,129780.0,4483.0,54443.0,2.34,3.58,41.95,47.03,141.0,0.08,0.91,62.50,MH,ATH,POWER
36,INDIAMART,4810.62,-53.52,51.0,H-SC,5.24,129537.0,6201.0,125431.0,-0.14,5.03,96.83,106.72,119.0,0.05,0.91,29.58,AR,ATH,MISC
31,HINDZINC,730.22,27.24,49.0,M-LC,9.32,207495.0,2419.0,110159.0,0.53,1.18,53.09,54.89,52.0,0.02,1.46,25.56,X5K,ATH,METALS
86,WIPRO,420.00,-16.06,40.0,M-LC,5.64,147256.0,-3689.0,113137.0,0.43,-2.44,76.83,72.51,53.0,-0.03,1.04,3.43,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TATAMOTORS,1065.00,-53.17,22.0,X-LC,34.23,147269.0,-129254.0,239327.0,-0.53,-46.74,162.51,39.81,54.0,-0.54,1.04,3.80,XY24,NTT,AUTO
23,DMART,5391.45,-21.31,25.0,X-LC,7.16,200550.0,-9857.0,69029.0,0.00,-4.68,34.42,28.12,38.0,-0.14,1.41,17.83,X40N,ATH,FMCG
44,JSWINFRA,342.00,-24.57,26.0,X-MC,4.43,189371.0,-10976.0,41472.0,0.12,-5.48,21.90,15.23,178.0,-0.26,1.33,25.01,X40N,NTT,REALTY
12,BATAINDIA,2096.00,-40.87,31.0,X-SC,8.09,88395.0,-40275.0,85575.0,0.29,-31.30,96.81,35.20,219.0,-0.47,0.62,1.46,X40,NTT,FOOTWEAR
30,HINDUNILVR,2922.00,-14.27,31.0,X-LC,6.22,240500.0,-16563.0,51708.0,-0.37,-6.44,21.50,13.67,24.0,-0.32,1.69,13.54,XY25,NTT,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,PGHH,17907.65,-31.74,33.0,X-MC,0.29,197610.0,-3210.0,71001.0,0.00,-1.60,35.93,33.76,80.0,-0.05,1.39,3.52,X40,ATH,FMCG
41,ITC,452.00,-38.77,43.0,X-LC,0.82,197947.0,-2191.0,23081.0,0.19,-1.09,11.66,10.44,4.0,-0.09,1.39,4.41,X40,NTT,FMCG
66,SIEMENS,4671.50,-5.61,37.0,H-LC,1.58,151845.0,-34250.0,81723.0,0.12,-18.40,53.82,25.51,15.0,-0.42,1.07,11.90,AR,ATH,ELECTRICAL
28,HAVELLS,2069.16,-8.59,37.0,X-MC,3.30,304920.0,-23071.0,129591.0,0.24,-7.03,42.50,32.48,92.0,-0.18,2.15,3.59,X40,ATH,ELECTRICAL
80,UNITDSPR,1644.00,-6.24,65.0,X-LC,3.78,245960.0,11816.0,36820.0,0.06,5.05,14.97,20.77,86.0,0.32,1.73,12.31,X40N,NTT,BREWERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,PAGEIND,51769.94,-29.60,33.0,X-MC,7.57,79500.0,-4010.0,24041.0,0.03,-4.80,30.24,23.99,82.0,-0.17,0.56,0.74,X40,ATH,APPARELS
1,ABB,7934.00,-42.98,32.0,H-LC,4.96,238272.0,-23347.0,142558.0,-1.05,-8.92,59.83,45.57,7.0,-0.16,1.68,1.39,AR,NTT,ELECTRICAL
12,BATAINDIA,2096.00,-40.87,31.0,X-SC,8.09,88395.0,-40275.0,85575.0,0.29,-31.30,96.81,35.20,219.0,-0.47,0.62,1.46,X40,NTT,FOOTWEAR
53,QUESS,986.00,-49.86,36.0,X-SC,37.31,49612.0,-15394.0,166319.0,-0.00,-23.68,335.24,232.18,198.0,-0.09,0.35,1.54,XY24,NTT,MISC
10,BAJAJHFL,181.50,-16.38,35.0,X-MC,6.35,176333.0,-24173.0,121335.0,-1.81,-12.06,68.81,48.45,90.0,-0.20,1.24,1.68,X40N,ATH,FINANCE


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-49.86,36.0,X-SC,37.31,49612.0,-15394.0,166319.0,-0.00,-23.68,335.24,232.18,198.0,-0.09,0.35,1.54,XY24,NTT,MISC
51,PAGEIND,51769.94,-29.60,33.0,X-MC,7.57,79500.0,-4010.0,24041.0,0.03,-4.80,30.24,23.99,82.0,-0.17,0.56,0.74,X40,ATH,APPARELS
57,RELAXO,1176.00,-40.27,43.0,X-SC,4.18,81385.0,-63775.0,139705.0,0.34,-43.93,171.66,52.31,136.0,-0.46,0.57,8.94,X40N,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-11.87,37.0,X-MC,3.12,86970.0,-3708.0,18612.0,0.71,-4.09,21.40,16.44,101.0,-0.20,0.61,15.26,X40,ATH,PHARMA
12,BATAINDIA,2096.00,-40.87,31.0,X-SC,8.09,88395.0,-40275.0,85575.0,0.29,-31.30,96.81,35.20,219.0,-0.47,0.62,1.46,X40,NTT,FOOTWEAR


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TCS,4389.96,-29.33,46.0,X-LC,9.60,281558.0,-64398.0,131093.0,0.12,-18.61,46.56,19.28,1.0,-0.49,1.98,4.09,X40,ATH,IT
39,INFY,2275.00,-18.52,55.0,X-LC,4.90,317583.0,4695.0,166985.0,0.96,1.50,52.58,54.87,3.0,0.03,2.24,9.91,X40,BTT,IT
41,ITC,452.00,-38.77,43.0,X-LC,0.82,197947.0,-2191.0,23081.0,0.19,-1.09,11.66,10.44,4.0,-0.09,1.39,4.41,X40,NTT,FMCG
83,VBL,671.64,-16.88,52.0,X-LC,5.64,299222.0,-16620.0,129294.0,-0.26,-5.26,43.21,35.67,5.0,-0.13,2.11,7.86,X40N,ATH,FMCG
1,ABB,7934.00,-42.98,32.0,H-LC,4.96,238272.0,-23347.0,142558.0,-1.05,-8.92,59.83,45.57,7.0,-0.16,1.68,1.39,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,COFFEEDAY,80.00,-49.70,35.0,L-SC,27.28,74650.0,-38899.0,78950.0,-0.28,-34.26,105.76,35.27,268.0,-0.49,0.53,82.71,XR,NTT,HOTELS
7,ASIANTILES,137.00,7383.33,64.0,L-SC,13.00,82104.0,-11706.0,88327.0,0.41,-12.48,107.58,81.67,269.0,-0.13,0.58,59.00,XR,NTT,CERAMICS
49,MASFIN,398.61,-17.80,53.0,H-SC,8.00,93300.0,-4680.0,26283.0,1.19,-4.78,28.17,22.05,152.0,-0.18,0.66,35.62,XR,ATH,FINANCE
85,WHIRLPOOL,2270.00,-40.16,54.0,M-SC,7.14,124154.0,9356.0,84685.0,0.71,8.15,68.21,81.92,223.0,0.11,0.87,46.98,XR,NTT,DURABLES
46,KPIGREEN,731.05,9.02,57.0,H-SC,9.73,129780.0,4483.0,54443.0,2.34,3.58,41.95,47.03,141.0,0.08,0.91,62.50,MH,ATH,POWER


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,VAIBHAVGBL,521.00,74.10,65.0,H-SC,4.92,151067.0,-31708.0,142773.0,0.75,-17.35,94.51,60.77,125.0,-0.22,1.06,36.05,XR,NTT,JEWELLERY
68,SONACOMS,806.63,-30.87,70.0,M-SC,6.25,86012.0,-15147.0,55151.0,0.08,-14.97,64.12,39.54,202.0,-0.27,0.61,21.29,AR,ATH,AUTO
46,KPIGREEN,731.05,9.02,57.0,H-SC,9.73,129780.0,4483.0,54443.0,2.34,3.58,41.95,47.03,141.0,0.08,0.91,62.50,MH,ATH,POWER
5,ANGELONE,3033.00,21.43,67.0,X-SC,6.44,215004.0,23998.0,33713.0,0.21,12.56,15.68,30.21,157.0,0.71,1.51,36.07,X40N,NTT,FINANCE
7,ASIANTILES,137.00,7383.33,64.0,L-SC,13.00,82104.0,-11706.0,88327.0,0.41,-12.48,107.58,81.67,269.0,-0.13,0.58,59.00,XR,NTT,CERAMICS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.69
1,25,44.33
2,50,75.41


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.87
LC    33.08
MC    22.99
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.00
X40      16.28
X40N     12.44
XY25     11.90
XR        9.88
AR        7.77
OX40N     6.50
X200      1.77
X5K       1.46
SR        1.03
MH        0.91
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.21
X-LC    22.42
X-MC    18.64
M-SC    12.62
X-SC     5.59
M-LC     5.16
H-LC     4.52
H-MC     2.31
M-MC     1.72
L-SC     1.45
L-LC     0.98
L-MC     0.32
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.19,-4.90,38.82
IT,12.45,-18.89,82.52
FINANCE,12.19,-9.20,58.70
MISC,7.81,-17.25,76.39
ELECTRICAL,5.67,-12.32,52.14
PAINTS,5.56,-16.78,34.19
BANKS,4.05,-17.56,85.95
INSURANCE,3.91,0.36,36.78
AUTO,2.71,-49.73,112.22


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3313813.0,22
XR,1254135.0,13
AR,1185251.0,9
X40,844047.0,12
X40N,700262.0,9
OX40N,562972.0,9
XY25,484726.0,8
SR,274468.0,2
X5K,110159.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3391187.0,24
M-SC,1252916.0,15
X-MC,1227390.0,14
X-LC,1094115.0,13
X-SC,554206.0,6
H-LC,296628.0,3
M-LC,277158.0,4
H-MC,259688.0,2
L-SC,256992.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1191420.0      6
           AR         838058.0      5
           XR         782098.0      7
M-SC       XY24       722151.0      6
X-MC       XY24       578224.0      4
X-LC       X40        459540.0      5
           XY24       313506.0      2
X-SC       X40N       302312.0      4
X-MC       X40        298932.0      6
H-SC       SR         274468.0      2
           OX40N      250700.0      3
X-LC       X40N       235143.0      3
H-LC       AR         224281.0      2
X-MC       XY25       187427.0      2
H-MC       XY24       184147.0      1
L-SC       XR         167277.0      2
X-SC       XY24       166319.0      1
X-MC       X40N       162807.0      2
M-MC       XY24       148600.0      1
M-SC       OX40N      147016.0      4
           XR         134110.0      2
           XY25       126727.0      1
           AR         122912.0      2
M-LC       XR         113137.0      1
           X5K        110159.0      1
L-SC       OX40N       89715.0      1
X-LC       XY25        85926.0      3
X-SC       X40         85575.0      1
H-MC       OX40N       75541.0      1
H-LC       X200        72347.0      1
L-MC       XR          57513.0      1
H-SC       MH          54443.0      1
M-LC       XY25        44416.0      1
L-LC       XY25        40230.0      1
M-LC       XY24         9446.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 30.0 seconds
